# Guide: Using Vector Store Index with Existing Weaviate Vector Store

In [1]:
import weaviate

In [2]:
client = weaviate.Client("https://test-cluster-bbn8vqsn.weaviate.network")

## Prepare Sample "Existing" Weaviate Vector Store

### Define schema
We create a schema for "Book" class, with 4 properties: title (str), author (str), content (str), and year (int)

In [28]:
try:
    client.schema.delete_class("Book")
except:
    pass

In [30]:
schema = {
    "classes": [
        {
            "class": "Book",
            "properties": [
                {"name": "title", "dataType": ["text"]},
                {"name": "author", "dataType": ["text"]},
                {"name": "content", "dataType": ["text"]},
                {"name": "year", "dataType": ["int"]},
            ],
        },
    ]
}

if not client.schema.contains(schema):
    client.schema.create(schema)

### Define sample data
We create 4 sample books 

In [31]:
books = [
    {
        "title": "To Kill a Mockingbird",
        "author": "Harper Lee",
        "content": "To Kill a Mockingbird is a novel by Harper Lee published in 1960...",
        "year": 1960,
    },
    {
        "title": "1984",
        "author": "George Orwell",
        "content": "1984 is a dystopian novel by George Orwell published in 1949...",
        "year": 1949,
    },
    {
        "title": "The Great Gatsby",
        "author": "F. Scott Fitzgerald",
        "content": "The Great Gatsby is a novel by F. Scott Fitzgerald published in 1925...",
        "year": 1925,
    },
    {
        "title": "Pride and Prejudice",
        "author": "Jane Austen",
        "content": "Pride and Prejudice is a novel by Jane Austen published in 1813...",
        "year": 1813,
    },
]

### Add data
We add the sample books to our Weaviate "Book" class (with embedding of content field

In [32]:
from llama_index.embeddings.openai import OpenAIEmbedding

embed_model = OpenAIEmbedding()

In [33]:
with client.batch as batch:
    for book in books:
        vector = embed_model.get_text_embedding(book["content"])
        batch.add_data_object(data_object=book, class_name="Book", vector=vector)

## Query Against "Existing" Weaviate Vector Store 

In [12]:
from llama_index.vector_stores import WeaviateVectorStore
from llama_index import VectorStoreIndex
from llama_index.response.pprint_utils import pprint_source_node

You must properly specify a "index_name" that matches the desired Weaviate class and select a class property as the "text" field.

In [ ]:
vector_store = WeaviateVectorStore(
    weaviate_client=client, index_name="Book", text_key="content"
)

In [14]:
retriever = VectorStoreIndex.from_vector_store(vector_store).as_retriever(
    similarity_top_k=1
)

In [15]:
nodes = retriever.retrieve("What is that book about a bird again?")

Let's inspect the retrieved node. We can see that the book data is loaded as LlamaIndex `Node` objects, with the "content" field as the main text.

In [22]:
pprint_source_node(nodes[0])

Document ID: cf927ce7-0672-4696-8aae-7e77b33b9659
Similarity: None
Text: author: Harper Lee title: To Kill a Mockingbird year: 1960  To
Kill a Mockingbird is a novel by Harper Lee published in 1960......


The remaining fields should be loaded as metadata (in `metadata`)

In [30]:
nodes[0].node.metadata

{'author': 'Harper Lee', 'title': 'To Kill a Mockingbird', 'year': 1960}